In [247]:
import requests
import json
from datetime import date
import pandas as pd
%pip install scikit-learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier
%pip install requests


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [248]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [249]:
df = df.sort_values("date")

In [250]:
df = df.reset_index(drop=True)

In [251]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [252]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target)


/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/3228710534.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/3228710534.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/3228710534.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [253]:
df["target"][pd.isnull(df["target"])] = 2

/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/1618397414.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/161839

In [254]:
df["target"] = df["target"].astype(int,errors="ignore")

In [255]:
df["won"].value_counts()

won
True     10006
False    10006
Name: count, dtype: int64

In [256]:
df["target"].value_counts()

target
1    9992
0    9990
2      30
Name: count, dtype: int64

In [257]:
nulls = pd.isnull(df)

In [258]:
nulls = nulls.sum()

In [259]:
nulls = nulls[nulls> 0]

In [260]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [261]:
valid_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object', length=140)

In [262]:
df = df[valid_columns].copy()

In [263]:
rr = RidgeClassifier(alpha = 1)
split = TimeSeriesSplit(n_splits = 3)

sfs = SequentialFeatureSelector(rr,n_features_to_select = 30, direction="forward", cv = split )

In [264]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [265]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [266]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [267]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [268]:
predictors = list(selected_columns[sfs.get_support()])

In [269]:
predictors

['mp',
 'usg%',
 'ft_max',
 'blk_max',
 'tov_max',
 'trb%_max',
 'tov%_max',
 'usg%_max',
 'home',
 'mp_opp',
 'fg%_opp',
 'ft%_opp',
 'orb_opp',
 'ast_opp',
 'blk_opp',
 'pf_opp',
 'efg%_opp',
 'ftr_opp',
 'blk%_opp',
 'usg%_opp',
 '3p_max_opp',
 '3p%_max_opp',
 'ft%_max_opp',
 'pf_max_opp',
 'pts_max_opp',
 'drb%_max_opp',
 'ast%_max_opp',
 'stl%_max_opp',
 'blk%_max_opp',
 'home_opp']

In [270]:
def backtest(data, model, predictors, start = 2, step = 1 ):
    all_predictions = []

    seasons = sorted(data["season"].unique())

    for i in range(start, len(seasons), step):
        season = seasons[i]

        train = data[data["season"] < season]
        test = data[data["season"] == season]

        model.fit(train[predictors], train["target"])

        preds = model.predict(test[predictors])

        preds = pd.Series(preds, index = test.index)

        combined = pd.concat([test["target"], preds], axis = 1)
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [271]:
predictions = backtest(df, rr, predictors)

In [272]:
predictions

,actual,prediction
5156,0,0
5157,1,0
5158,1,1
5159,1,0
5160,0,1
...,...,...
20007,1,1
20008,0,0
20009,1,1
20010,2,1


In [273]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions["actual"] != 2]

accuracy_score(predictions["actual"], predictions["prediction"])

0.5402670983407527

In [274]:
df.groupby("home").apply(lambda x: x[x["won"] == 1 ].shape[0] /x.shape[0])

/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/3236211030.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("home").apply(lambda x: x[x["won"] == 1 ].shape[0] /x.shape[0])


home
0.0    0.427543
1.0    0.572457
dtype: float64

In [275]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

In [276]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]


In [277]:
def find_team_averages(team):
    # Select only the numeric columns for rolling
    numeric_cols = team.select_dtypes(include=['number'])
    rolling = numeric_cols.rolling(10).mean()
    # Now, you want to merge back the non-numeric data which was excluded
    non_numeric_cols = team.select_dtypes(exclude=['number'])
    return pd.concat([rolling, non_numeric_cols], axis=1)



In [278]:
df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/378916889.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [279]:
df_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,season,won,team
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,GSW
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NOP
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,CHI
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,DET
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20007,0.0,0.521739,0.357353,0.576794,0.379310,0.375758,0.444893,0.418605,0.352381,0.785998,...,0.0710,0.314361,0.340897,0.411848,0.642353,0.390179,0.5,2023.0,True,DEN
20008,0.0,0.419565,0.370588,0.439952,0.420690,0.406061,0.469121,0.316279,0.260317,0.829988,...,0.0910,0.262998,0.246154,0.490521,0.535294,0.366071,0.5,2023.0,False,MIA
20009,0.0,0.497826,0.342647,0.560526,0.403448,0.384848,0.466508,0.416279,0.349206,0.788681,...,0.0883,0.319811,0.340769,0.425118,0.628235,0.359821,0.5,2023.0,True,DEN
20010,0.0,0.400000,0.376471,0.412919,0.420690,0.419697,0.458551,0.306977,0.255556,0.821587,...,0.1108,0.257547,0.237949,0.512322,0.512941,0.356250,0.5,2023.0,False,MIA


In [280]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df,df_rolling], axis = 1)

In [281]:
df = df.dropna()

In [282]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df,"home")
df["team_opp_next"] = add_col(df,"team_opp")
df["date_next"] = add_col(df,"date")


/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/3261750475.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/8d/ycl0fys12wq90tgn83r138pw0000gn/T/ipykernel_64117/3261750475.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(

In [283]:
full = df.merge(
    df[rolling_cols + ["team_opp_next", "date_next", "team"]], 
    left_on = ["team", "date_next"], 
    right_on = ["team_opp_next", "date_next"]
)

In [284]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [285]:
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_10_x',
 'team_opp_next_x',
 'date_next',
 'team_10_y',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [286]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [287]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [288]:
predictors = list(selected_columns[sfs.get_support()])

In [289]:
predictions = backtest(full, rr, predictors)

In [290]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.6379336579427876

random forest classifier
xgboost

ridge regression model

backward feature selection

different ratios of games for rolling averages maybe to look at last 10 - 5 games to see if htey are on a hot streak

top predict future rows: 
webscrape to get the 
need to fill in home next opp and the date they are going to play
merge and include rows
rerun predictions
we want to look at rows were target is two which means it is a future games and find by looking for the 2 



In [291]:
# Get the current date
current_date = date.today()

# Format the date as YYYY/MM/DD
formatted_date = current_date.strftime('%Y/%m/%d')

url = f"https://api.sportradar.com/nba/trial/v8/en/games/{formatted_date}/schedule.json?api_key=Ux0gftPqB53SqaH5JBtp6arHMvYs6HaIy1NvsK00"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)
game = response.json()

games_data = game['games']

df_teams_time = pd.DataFrame([{
    'Home Team': game['home']['name'],
    ''
    'Away Team': game['away']['name'],
    'Scheduled Time': game['scheduled']

} for game in games_data])

print(df_teams_time)

                Home Team          Away Team        Scheduled Time
0         New York Knicks      Brooklyn Nets  2024-03-23T17:00:00Z
1           Orlando Magic   Sacramento Kings  2024-03-23T23:00:00Z
2           Atlanta Hawks  Charlotte Hornets  2024-03-23T23:30:00Z
3      Washington Wizards    Toronto Raptors  2024-03-24T00:00:00Z
4           Chicago Bulls     Boston Celtics  2024-03-24T00:00:00Z
5       San Antonio Spurs       Phoenix Suns  2024-03-24T00:00:00Z
6         Houston Rockets          Utah Jazz  2024-03-24T00:00:00Z
7  Portland Trail Blazers     Denver Nuggets  2024-03-24T02:00:00Z
